# BAN427 home exam

## Modules

In [25]:
# Modules
import pandas as pd
import os
import numpy

In [ ]:
df.to_csv("exam_case_data.csv")

In [ ]:
master = pd.read_excel("exam_case_data.xlsx")
df = master.copy()

In [ ]:
#df["TIME2"].head(50)



df.loc[df["TIME2"] != 2, "FULL_CHURN"] = 1

df['FULL_CHURN'] = np.where(df['TIME2'] != 2, 1, 0)


df

pd.set_option("display.max_rows", 100)

In [ ]:
df.head()

In [ ]:
df.iloc[50:200,:]

In [ ]:
df.loc[50:100,:]

In [ ]:
len(df.FULL_CHURN)-df.FULL_CHURN.astype('category').describe().freq
df['PARTIAL_CHURN'] = np.where((df['NUMBER_COVERS_TIME2'] - df['NUMBER_COVERS_TIME1']) < 0, 1, 0)
df.head(50)